<a href="https://colab.research.google.com/github/rahibahmed/Assignment1DataCollection/blob/main/Assignment2DataCollectionAndCuration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update #update linux
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #download and install openjdk
!wget -q http://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz #download spark binary (gunzip). -q: Turn off Wget’s output.
!tar xf spark-3.2.1-bin-hadoop2.7.tgz #extract the spark package
!pip install -q findspark #install the findspark package

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net

In [ ]:
#set environment variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [ ]:
# From Spark 2.0, SparkSession provides a common entry point for a Spark application
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark.conf.set('spark.sql.shuffle.partitions', 6)
# spark.conf.set('spark.executor.memory', '2g')
spark

In [ ]:
!ls

sample_data  spark-3.2.1-bin-hadoop2.7	spark-3.2.1-bin-hadoop2.7.tgz


In [ ]:
!ls sample_data  

anscombe.json		      mnist_test.csv	     POI.csv
california_housing_test.csv   mnist_train_small.csv  README.md
california_housing_train.csv  orders_data.zip


##UNZIP THE ORDERSDATA

In [ ]:
!unzip -q sample_data/orders_data.zip -d sample_data/orders_data

In [ ]:
!ls sample_data/orders_data

orders_100.csv	orders_28.csv  orders_46.csv  orders_64.csv  orders_82.csv
orders_10.csv	orders_29.csv  orders_47.csv  orders_65.csv  orders_83.csv
orders_11.csv	orders_2.csv   orders_48.csv  orders_66.csv  orders_84.csv
orders_12.csv	orders_30.csv  orders_49.csv  orders_67.csv  orders_85.csv
orders_13.csv	orders_31.csv  orders_4.csv   orders_68.csv  orders_86.csv
orders_14.csv	orders_32.csv  orders_50.csv  orders_69.csv  orders_87.csv
orders_15.csv	orders_33.csv  orders_51.csv  orders_6.csv   orders_88.csv
orders_16.csv	orders_34.csv  orders_52.csv  orders_70.csv  orders_89.csv
orders_17.csv	orders_35.csv  orders_53.csv  orders_71.csv  orders_8.csv
orders_18.csv	orders_36.csv  orders_54.csv  orders_72.csv  orders_90.csv
orders_19.csv	orders_37.csv  orders_55.csv  orders_73.csv  orders_91.csv
orders_1.csv	orders_38.csv  orders_56.csv  orders_74.csv  orders_92.csv
orders_20.csv	orders_39.csv  orders_57.csv  orders_75.csv  orders_93.csv
orders_21.csv	orders_3.csv   orders_58.csv  orders_7

##**1. IMPORT DATA DEFINED BY ORDER SCHEMA**

In [ ]:
from pyspark.sql.types import *

file_location = "sample_data/orders_data/orders_*.csv"

# create schema
ordersSchema = StructType([
  StructField("Order_ID", DoubleType(), True),
  StructField("Country", StringType(), True),
  StructField("Province", StringType(), True),
  StructField("City", StringType(), True),
  StructField("Latitude", DoubleType(), True),
  StructField("Longitude", DoubleType(), True),
  StructField("TimeStamp", StringType(), True),
  StructField("Sales_Volume", DoubleType(), True)
  ])

# create DataFrame 
data = (
  spark
    .read
    .schema(ordersSchema)
    .csv(file_location)
)


# create table for SQL analytics
data.createOrReplaceTempView("orders")

data.show()

+--------+-------+--------+--------------------+---------+---------+-------------------+------------+
|Order_ID|Country|Province|                City| Latitude|Longitude|          TimeStamp|Sales_Volume|
+--------+-------+--------+--------------------+---------+---------+-------------------+------------+
|231542.0| Canada|      AB|             Calgary|-113.9835|-113.9389|2022/04/22 08:28:49|       41.49|
|473450.0| Canada|      AB|            Edmonton|-113.4467|-113.3654|2022/04/22 05:04:24|       48.52|
|376604.0| Canada|      AB|        Medicine Hat|-110.5798|-110.4884|2022/04/22 18:14:14|       60.79|
|440105.0| Canada|      AB|       Sherwood Park|-113.2427| -113.242|2022/04/22 11:27:20|       77.81|
|483058.0| Canada|      AB|            Beaumont|-113.3783|-113.2894|2022/04/22 21:04:24|       12.06|
|303271.0| Canada|      AB|            Edmonton|-113.3609|-113.3519|2022/04/22 06:32:45|        99.0|
|444942.0| Canada|      AB|       Fort Mcmurray|-111.4019|-111.4018|2022/04/22 10:

###**Creating Data Stream for ordersSchema**

In [ ]:
from pyspark.sql.functions import *

# create streaming DF
streamingDF = (
  spark
    .readStream                       
    .schema(ordersSchema)              # schema for the streaming DF
    .option("maxFilesPerTrigger", 1)  # Form a stream by triggering one file at a time
    .csv(file_location)
)

# Is it a streaming DF?
print(streamingDF.isStreaming)

True


###**Creating temp view for dataframe**

In [ ]:
streamingDF.createOrReplaceTempView("ordersSQL")

###**Unique Cities in the dataset**

In [ ]:
uniqueCity = spark.sql("SELECT count(distinct city) as UniqueCity FROM all_orders")
uniqueCity.show()

+----------+
|UniqueCity|
+----------+
|       619|
+----------+



###**Querying the dataframe**

In [ ]:
streamVol = spark.sql("SELECT Province, TimeStamp from ordersSQL GROUP BY Province, TimeStamp ORDER BY TimeStamp DESC")

###**Write Stream to display query results**

In [ ]:
query = (
  streamVol
    .writeStream
    .format("memory")        # store in-memory table
    .queryName("orders_k")     
    .outputMode("complete")  
    .start()
)

##**2. Latest time of the order that was made in the province**

In [ ]:
spark.sql("SELECT Province, MAX(TimeStamp) TimeStamp FROM orders_k GROUP BY Province").show()

+--------+-------------------+
|Province|          TimeStamp|
+--------+-------------------+
|      AB|2022/04/23 01:13:04|
|      BC|2022/04/23 01:05:14|
|      MB|2022/04/23 00:25:42|
|      NB|2022/04/23 00:47:52|
|      NL|2022/04/23 00:36:20|
|      NS|2022/04/23 00:19:17|
|      NT|2022/04/22 17:33:18|
|      ON|2022/04/23 01:18:38|
|      PE|2022/04/22 23:23:21|
|      QC|2022/04/23 01:17:12|
|      SK|2022/04/23 01:19:30|
|      YT|2022/04/22 20:48:47|
+--------+-------------------+



###Query Check 1 for province YT

In [ ]:
spark.sql("SELECT Province, TimeStamp FROM orders_k Where Province='YT' order by TimeStamp DESC").show()

+--------+-------------------+
|Province|          TimeStamp|
+--------+-------------------+
|      YT|2022/04/22 20:48:47|
|      YT|2022/04/22 07:53:22|
|      YT|2022/04/22 05:09:08|
+--------+-------------------+



###Query Check 1 for province AB

In [ ]:
spark.sql("SELECT Province, TimeStamp FROM orders_k Where Province='AB' order by TimeStamp DESC").show(5)

+--------+-------------------+
|Province|          TimeStamp|
+--------+-------------------+
|      AB|2022/04/23 01:13:04|
|      AB|2022/04/23 01:09:22|
|      AB|2022/04/23 01:07:05|
|      AB|2022/04/23 01:06:51|
|      AB|2022/04/23 01:02:53|
+--------+-------------------+
only showing top 5 rows



##**3. Import POI data set**

In [ ]:
from pyspark.sql.types import *

file_location_POI = "sample_data/POI.csv"

# create schema
poi = StructType([
  StructField("POI_ID", StringType(), True),
  StructField("Latitude_POI", DoubleType(), True),
  StructField("Longitude_POI", DoubleType(), True)
  ])
inferSchema = "true"
firstRowHeader = "true"
delimeter = ","
file_type = "csv"
# create DataFrame 
df_poi = (
  spark.read.load(file_location_POI,format=file_type, header=firstRowHeader, sep=delimeter, schema=poi)
)


# create table for SQL analytics
data.createOrReplaceTempView("poi")

data.show()

+--------+-------+--------+--------------------+---------+---------+-------------------+------------+
|Order_ID|Country|Province|                City| Latitude|Longitude|          TimeStamp|Sales_Volume|
+--------+-------+--------+--------------------+---------+---------+-------------------+------------+
|231542.0| Canada|      AB|             Calgary|-113.9835|-113.9389|2022/04/22 08:28:49|       41.49|
|473450.0| Canada|      AB|            Edmonton|-113.4467|-113.3654|2022/04/22 05:04:24|       48.52|
|376604.0| Canada|      AB|        Medicine Hat|-110.5798|-110.4884|2022/04/22 18:14:14|       60.79|
|440105.0| Canada|      AB|       Sherwood Park|-113.2427| -113.242|2022/04/22 11:27:20|       77.81|
|483058.0| Canada|      AB|            Beaumont|-113.3783|-113.2894|2022/04/22 21:04:24|       12.06|
|303271.0| Canada|      AB|            Edmonton|-113.3609|-113.3519|2022/04/22 06:32:45|        99.0|
|444942.0| Canada|      AB|       Fort Mcmurray|-111.4019|-111.4018|2022/04/22 10:

##**4. Distance between order and POI_ID**

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrameWriter as W
from math import radians, cos, sin, asin, sqrt

def get_distance(longit_a, latit_a, longit_b, latit_b):
# Transform to radians
  print("inside")
  longit_a, latit_a, longit_b, latit_b = map(radians, [longit_a,  latit_a, longit_b, latit_b])
  
  dist_longit = longit_b - longit_a
  dist_latit = latit_b - latit_a
  print(dist_latit)
# Calculate area
  area = sin(dist_latit/2)**2 + cos(latit_a) * cos(latit_b) * sin(dist_longit/2)**2
# Calculate the central angle
  central_angle = 2 * asin(sqrt(area))
  radius = 6371
# Calculate Distance
  distance = central_angle * radius
  return distance

In [ ]:
udf_get_distance = udf(get_distance)

In [ ]:
orders_dataset = spark.sql("SELECT * from ordersSQL")

In [ ]:
query = (
  orders_dataset
    .writeStream
    .format("memory")        # store in-memory table
    .queryName("all_orders")  
    .start()
)

In [ ]:
order_loc_pair = orders_dataset.crossJoin(df_poi)

In [ ]:
query = (
  order_loc_pair
    .writeStream
    .format("memory")        # store in-memory table
    .queryName("order_loc_join")  
    .start()
)

In [ ]:
spark.sql("SELECT * FROM order_loc_join").show(10)

+--------+-------+--------+-----------+--------+---------+-------------------+------------+------+------------+-------------+
|Order_ID|Country|Province|       City|Latitude|Longitude|          TimeStamp|Sales_Volume|POI_ID|Latitude_POI|Longitude_POI|
+--------+-------+--------+-----------+--------+---------+-------------------+------------+------+------------+-------------+
|456581.0| Canada|      ON|   Kingston|-76.4227| -76.3738|2022/04/22 03:25:20|       20.76| POI-1|   45.521629|   -74.085634|
|456581.0| Canada|      ON|   Kingston|-76.4227| -76.3738|2022/04/22 03:25:20|       20.76| POI-2|   53.596345|  -114.465122|
|456581.0| Canada|      ON|   Kingston|-76.4227| -76.3738|2022/04/22 03:25:20|       20.76| POI-3|   44.897823|   -62.987528|
|313471.0| Canada|      ON|     Oshawa|-78.8064| -78.7803|2022/04/22 02:11:24|       55.81| POI-1|   45.521629|   -74.085634|
|313471.0| Canada|      ON|     Oshawa|-78.8064| -78.7803|2022/04/22 02:11:24|       55.81| POI-2|   53.596345|  -114.

In [ ]:
order_poi_pair_distance = order_loc_pair.withColumn("ABS_DISTANCE", 
                                                    udf_get_distance(
                                                        order_loc_pair.Longitude, order_loc_pair.Latitude, 
                                                        order_loc_pair.Longitude_POI, order_loc_pair.Latitude_POI).cast(DoubleType()))

In [ ]:
query = (
  order_poi_pair_distance
    .writeStream
    .format("memory")        # store in-memory table
    .queryName("order_pair_poi_for")
    .start()
)

In [ ]:
spark.sql("SELECT * FROM order_pair_poi_for").show()

+--------+-------+--------+-----------+---------+---------+-------------------+------------+------+------------+-------------+------------------+
|Order_ID|Country|Province|       City| Latitude|Longitude|          TimeStamp|Sales_Volume|POI_ID|Latitude_POI|Longitude_POI|      ABS_DISTANCE|
+--------+-------+--------+-----------+---------+---------+-------------------+------------+------+------------+-------------+------------------+
|456581.0| Canada|      ON|   Kingston| -76.4227| -76.3738|2022/04/22 03:25:20|       20.76| POI-1|   45.521629|   -74.085634| 13560.57540942165|
|456581.0| Canada|      ON|   Kingston| -76.4227| -76.3738|2022/04/22 03:25:20|       20.76| POI-2|   53.596345|  -114.465122|14708.515156710147|
|456581.0| Canada|      ON|   Kingston| -76.4227| -76.3738|2022/04/22 03:25:20|       20.76| POI-3|   44.897823|   -62.987528|13523.975425249477|
|313471.0| Canada|      ON|     Oshawa| -78.8064| -78.7803|2022/04/22 02:11:24|       55.81| POI-1|   45.521629|   -74.08563

##**5. Shortest Distance between poi_id and order**

In [ ]:
shortest_dist_pair = spark.sql("Select Order_ID, Country, Province, City, TimeStamp, Sales_Volume, POI_ID, ABS_DISTANCE as shortestDistance 
    from (select Order_ID, Country, Province, City, TimeStamp, Sales_Volume, POI_ID, ABS_DISTANCE, 
          ROW_NUMBER() OVER(PARTITION BY Order_ID ORDER BY ABS_DISTANCE ASC) as rn from order_pair_poi_for) as a where rn = 1")

In [ ]:
shortest_dist_pair.show()

+--------+-------+--------+---------------+-------------------+------------+------+------------------+
|Order_ID|Country|Province|           City|          TimeStamp|Sales_Volume|POI_ID|  shortestDistance|
+--------+-------+--------+---------------+-------------------+------------+------+------------------+
|200041.0| Canada|      AB|       Edmonton|2022/04/22 02:04:24|       56.69| POI-3|16209.367269379594|
|200099.0| Canada|      ON|         Barrie|2022/04/22 22:02:27|       72.53| POI-3| 13889.52313484939|
|200137.0| Canada|      ON|      Brantford|2022/04/22 19:30:44|       32.78| POI-3|13954.071370976626|
|200171.0| Canada|      SK|      Saskatoon|2022/04/22 06:10:33|       81.83| POI-3|16167.766621680077|
|200183.0| Canada|      QC|       Montréal|2022/04/22 22:58:02|       56.63| POI-3|13188.188239030185|
|200231.0| Canada|      ON|        Toronto|2022/04/22 20:59:54|       77.48| POI-3|13857.486230055967|
|200242.0| Canada|      ON|          Maple|2022/04/22 21:46:03|       75.

In [ ]:
print(shortest_dist_pair.count())

19352


In [ ]:
spark.sql("SELECT COUNT(*) as total_orders from all_orders").show()

+------------+
|total_orders|
+------------+
|       20000|
+------------+



In [ ]:
spark.sql("SELECT COUNT(*) as total_pair from order_pair_poi_for").show()

+----------+
|total_pair|
+----------+
|     60000|
+----------+



##**6. For each POI, get the average, standard deviation and max of the (shortest) distances, as well as the count of the orders.**

In [ ]:
poi_1 = spark.sql("Select Order_ID, Country, Province, City, TimeStamp, Sales_Volume, POI_ID, ABS_DISTANCE as POI_1 from (select Order_ID, Country, Province, City, TimeStamp, Sales_Volume, POI_ID, ABS_DISTANCE, ROW_NUMBER() OVER(PARTITION BY Order_ID ORDER BY ABS_DISTANCE ASC) as rn from order_pair_poi_for) as a where rn = 2")

In [ ]:
poi_1.createOrReplaceTempView("poi1sql")
poi_1.show(10)

+--------+-------+--------+-----------+-------------------+------------+------+------------------+
|Order_ID|Country|Province|       City|          TimeStamp|Sales_Volume|POI_ID|             POI_1|
+--------+-------+--------+-----------+-------------------+------------+------+------------------+
|200021.0| Canada|      AB|    Calgary|2022/04/22 03:54:48|       28.84| POI-1|16739.114815958008|
|200026.0| Canada|      ON|Mississauga|2022/04/22 18:07:48|       47.73| POI-1|13933.687819594936|
|200041.0| Canada|      AB|   Edmonton|2022/04/22 02:04:24|       56.69| POI-1|16735.547913862214|
|200099.0| Canada|      ON|     Barrie|2022/04/22 22:02:27|       72.53| POI-1|13922.339285128917|
|200137.0| Canada|      ON|  Brantford|2022/04/22 19:30:44|       32.78| POI-1|13986.676376570755|
|200154.0| Canada|      ON|  Etobicoke|2022/04/22 03:23:39|       47.43| POI-1|13906.197189891822|
|200171.0| Canada|      SK|  Saskatoon|2022/04/22 06:10:33|       81.83| POI-1|16512.932512404972|
|200183.0|

In [ ]:
poi_2 = spark.sql("Select Order_ID, Country, Province, City, TimeStamp, Sales_Volume, POI_ID, ABS_DISTANCE as POI_2 from (select Order_ID, Country, Province, City, TimeStamp, Sales_Volume, POI_ID, ABS_DISTANCE, ROW_NUMBER() OVER(PARTITION BY Order_ID ORDER BY ABS_DISTANCE ASC) as rn from order_pair_poi_for) as a where rn = 3")

In [ ]:
poi_2.createOrReplaceTempView("poi2sql")
poi_2.show(10)

+--------+-------+--------+-----------+-------------------+------------+------+------------------+
|Order_ID|Country|Province|       City|          TimeStamp|Sales_Volume|POI_ID|             POI_2|
+--------+-------+--------+-----------+-------------------+------------+------+------------------+
|200021.0| Canada|      AB|    Calgary|2022/04/22 03:54:48|       28.84| POI-2|18637.666676177498|
|200026.0| Canada|      ON|Mississauga|2022/04/22 18:07:48|       47.73| POI-2|14993.946591700058|
|200041.0| Canada|      AB|   Edmonton|2022/04/22 02:04:24|       56.69| POI-2|18598.350544352586|
|200099.0| Canada|      ON|     Barrie|2022/04/22 22:02:27|       72.53| POI-2|14985.641290652007|
|200137.0| Canada|      ON|  Brantford|2022/04/22 19:30:44|       32.78| POI-2|15035.657605067863|
|200154.0| Canada|      ON|  Etobicoke|2022/04/22 03:23:39|       47.43| POI-2| 14972.50774136635|
|200171.0| Canada|      SK|  Saskatoon|2022/04/22 06:10:33|       81.83| POI-2|17780.899294918105|
|200183.0|

In [ ]:
poi_3 = spark.sql("Select Order_ID, Country, Province, City, TimeStamp, Sales_Volume, POI_ID, ABS_DISTANCE as POI_3 from (select Order_ID, Country, Province, City, TimeStamp, Sales_Volume, POI_ID, ABS_DISTANCE, ROW_NUMBER() OVER(PARTITION BY Order_ID ORDER BY ABS_DISTANCE ASC) as rn from order_pair_poi_for) as a where rn = 1")

In [ ]:
poi_3.createOrReplaceTempView("poi3sql")
poi_3.show(10)

+--------+-------+--------+-----------+-------------------+------------+------+------------------+
|Order_ID|Country|Province|       City|          TimeStamp|Sales_Volume|POI_ID|             POI_3|
+--------+-------+--------+-----------+-------------------+------------+------+------------------+
|200021.0| Canada|      AB|    Calgary|2022/04/22 03:54:48|       28.84| POI-3|16204.430710524315|
|200026.0| Canada|      ON|Mississauga|2022/04/22 18:07:48|       47.73| POI-3|13901.071108403206|
|200041.0| Canada|      AB|   Edmonton|2022/04/22 02:04:24|       56.69| POI-3|16209.367269379594|
|200099.0| Canada|      ON|     Barrie|2022/04/22 22:02:27|       72.53| POI-3| 13889.52313484939|
|200137.0| Canada|      ON|  Brantford|2022/04/22 19:30:44|       32.78| POI-3|13954.071370976626|
|200154.0| Canada|      ON|  Etobicoke|2022/04/22 03:23:39|       47.43| POI-3|13873.493500468629|
|200171.0| Canada|      SK|  Saskatoon|2022/04/22 06:10:33|       81.83| POI-3|16167.766621680077|
|200183.0|

In [ ]:
described_POI_1=poi_1.describe("POI_1")

In [ ]:
described_POI_2 = poi_2.describe("POI_2")

In [ ]:
described_POI_3=poi_3.describe("POI_3")

In [ ]:
described_POI_1.show()
described_POI_2.show()
described_POI_3.show()

+-------+------------------+
|summary|             POI_1|
+-------+------------------+
|  count|             19352|
|   mean|15109.827047396564|
| stddev|1530.6776408920375|
|    min|1712.1563208646014|
|    max|16740.642478696762|
+-------+------------------+

+-------+------------------+
|summary|             POI_2|
+-------+------------------+
|  count|             19352|
|   mean|16560.579406317684|
| stddev|1900.7086609756088|
|    min|3338.7477498065414|
|    max|19362.138962533838|
+-------+------------------+

+-------+------------------+
|summary|             POI_3|
+-------+------------------+
|  count|             19352|
|   mean| 14849.40713889063|
| stddev|1311.4081868194046|
|    min| 1213.195025859299|
|    max|16235.167523975275|
+-------+------------------+



In [ ]:
maxCountShort1 = spark.sql("SELECT POI_1, COUNT(POI_1) as countMax from poi1sql GROUP BY POI_1 HAVING countMax>1 ORDER BY countMax DESC")
maxCountShort1.show()

+------------------+--------+
|             POI_1|countMax|
+------------------+--------+
|13894.019384039946|       2|
| 16731.98131773307|       2|
|16738.054640564547|       2|
|16733.607025889512|       2|
|16737.923787491272|       2|
|13895.929765918796|       2|
|13893.159582511029|       2|
|16736.571168994324|       2|
|13884.328685576496|       2|
+------------------+--------+



In [ ]:
maxCountShort2 = spark.sql("SELECT POI_2, COUNT(POI_2) as countMax from poi2sql GROUP BY POI_2 HAVING countMax>1 ORDER BY countMax DESC")
maxCountShort2.show()

+------------------+--------+
|             POI_2|countMax|
+------------------+--------+
|18634.829351109773|       2|
|18639.777675442947|       2|
|14962.189412526426|       2|
|14955.949261189555|       2|
|18566.090157244027|       2|
|14963.135053134714|       2|
| 14964.86847198125|       2|
| 18641.05685041603|       2|
|18569.980022897224|       2|
+------------------+--------+



In [ ]:
maxCountShort3 = spark.sql("SELECT POI_3, COUNT(POI_3) as countMax from poi3sql GROUP BY POI_3 HAVING countMax>1 ORDER BY countMax DESC")
maxCountShort3.show()

+------------------+--------+
|             POI_3|countMax|
+------------------+--------+
|13860.455651864271|       2|
|16200.975022242168|       2|
|16214.534265161397|       2|
|16202.696999054095|       2|
|13861.235134899494|       2|
|13863.078538739743|       2|
|13851.395700546907|       2|
|16211.925880285315|       2|
|16203.929747694508|       2|
+------------------+--------+



In [ ]:
maxCountShort1.show()
maxCountShort2.show()
maxCountShort3.show()

+------------------+--------+
|             POI_1|countMax|
+------------------+--------+
|13894.019384039946|       2|
| 16731.98131773307|       2|
|16738.054640564547|       2|
|16733.607025889512|       2|
|16737.923787491272|       2|
|13895.929765918796|       2|
|13893.159582511029|       2|
|16736.571168994324|       2|
|13884.328685576496|       2|
+------------------+--------+

+------------------+--------+
|             POI_2|countMax|
+------------------+--------+
|18634.829351109773|       2|
|18639.777675442947|       2|
|14962.189412526426|       2|
|14955.949261189555|       2|
|18566.090157244027|       2|
|14963.135053134714|       2|
| 14964.86847198125|       2|
| 18641.05685041603|       2|
|18569.980022897224|       2|
+------------------+--------+

+------------------+--------+
|             POI_3|countMax|
+------------------+--------+
|13860.455651864271|       2|
|16200.975022242168|       2|
|16214.534265161397|       2|
|16202.696999054095|       2|
|13861.2

##**7. Based on max distance and order counts calculate the density**

In [ ]:
import numpy as np

import math
def calculateDensity(orderCount, maxDistance):
   density = orderCount/(math.pi*(maxDistance ** 2))
   return density

In [ ]:
orderCount = int(described_POI_1.toPandas().POI_1.values[0])
maxDistance=maxCountShort1.toPandas().POI_1.values
maxDistance = np.array(maxDistance)
maxDistance = maxDistance.astype(float)
minValue = np.amin(maxDistance)
density_poi_1 = calculateDensity(orderCount, minValue)

In [ ]:
orderCount = int(described_POI_2.toPandas().POI_2.values[0])
maxDistance=maxCountShort2.toPandas().POI_2.values
maxDistance = np.array(maxDistance)
maxDistance = maxDistance.astype(float)
minValue = np.amin(maxDistance)
density_poi_2 = calculateDensity(orderCount, minValue)

In [ ]:
orderCount = int(described_POI_3.toPandas().POI_3.values[0])
maxDistance=maxCountShort3.toPandas().POI_3.values
maxDistance = np.array(maxDistance)
maxDistance = maxDistance.astype(float)
minValue = np.amin(maxDistance)
density_poi_3 = calculateDensity(orderCount, minValue)

In [ ]:

print("Density of POI 1 =", density_poi_1)

print("Density of POI 2 =", density_poi_2)

print("Density of POI 3 =", density_poi_3)

Density of POI 1 = 2.7538990512244087e-05
Density of POI 2 = 2.7538990512244087e-05
Density of POI 3 = 3.210620026390737e-05
